In [83]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [84]:
a = pd.read_csv("/content/categorical_data.csv")
b = pd.read_csv("/content/continuous_data.csv")
b = b.dropna(axis=1)
c=  pd.read_csv("/content/Output_data.csv")
df = pd.read_csv("/content/Reduced_Features_set.csv")

In [85]:
a

,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22
0,4,3,0,0,0,0,0,0,1,0,0,0,0,2,0,0
1,4,2,0,1,0,0,0,0,0,0,0,0,0,2,0,0
2,4,1,0,0,0,0,0,1,1,0,0,0,0,3,0,0
3,3,3,0,0,0,0,0,1,1,0,0,0,0,3,1,0
4,4,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,3,2,0,0,0,0,0,0,1,0,0,0,0,2,0,0
583,3,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
584,3,1,0,0,1,0,0,1,0,1,0,0,0,4,0,0
585,3,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [86]:
b

,0,1,2,3,4,5,21,23,24
0,80,18,164,85,111.333333,21.258503,40,75.0,3827.40
1,80,21,140,60,86.666667,21.333333,33,44.0,451.00
2,78,20,166,98,120.666667,32.049948,33,39.0,857.46
3,81,19,156,70,98.666667,17.357667,33,41.0,1720.67
4,80,19,110,60,76.666667,19.531250,33,48.0,876.28
...,...,...,...,...,...,...,...,...,...
582,72,19,150,80,103.333333,20.666667,33,51.0,609.44
583,65,19,106,84,91.333333,26.835180,33,57.0,338.67
584,85,20,100,60,73.333333,20.811655,21,56.0,2174.08
585,100,22,110,80,90.000000,23.437500,29,46.0,531.07


In [44]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,lvef_class
0,0,80.0,18.0,164.0,85.0,111.333333,21.258503,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,40.0,0.0,75.0,3827.40,0
1,1,80.0,21.0,140.0,60.0,86.666667,21.333333,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,33.0,0.0,44.0,451.00,1
2,2,78.0,20.0,166.0,98.0,120.666667,32.049948,4.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,33.0,0.0,39.0,857.46,1
3,3,81.0,19.0,156.0,70.0,98.666667,17.357667,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,33.0,0.0,41.0,1720.67,1
4,4,80.0,19.0,110.0,60.0,76.666667,19.531250,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,33.0,0.0,48.0,876.28,1


In [46]:
Y = df['lvef_class'].to_numpy()

In [48]:
Y.shape

(587,)

#PCA on categorical values

In [87]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [101]:
for i in range(2,17):
  pca = PCA(n_components=i)
  principalComponents_i = pca.fit_transform(a)
  principalDf_i = pd.DataFrame(data = principalComponents_i)
  principalDf_i['0'] = b['0']
  principalDf_i['1'] = b['1']
  principalDf_i['2'] = b['2']
  principalDf_i['3'] = b['3']
  principalDf_i['4'] = b['4']
  principalDf_i['5'] = b['5']
  principalDf_i['21'] = b['21']
  principalDf_i['23'] = b['23']
  principalDf_i['24'] = b['24']
  Y = Y.astype(int)
  Y = Y.reshape(-1)
  x_train, x_test, y_train, y_test = train_test_split(principalDf_i,Y,test_size = 0.148)
  scaler = preprocessing.StandardScaler().fit(x_train)
  x_train = scaler.transform(x_train)
  scaler = preprocessing.StandardScaler().fit(x_test)
  x_test = scaler.transform(x_test)
  from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
  n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
  max_features = ['auto', 'sqrt']
  max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
  max_depth.append(None)
  min_samples_split = [2, 3,5]
  min_samples_leaf = [ 2, 3, 4]
  bootstrap = [True, False]# Create the random grid
  random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
  clf = RandomForestClassifier(n_estimators=1333,bootstrap=True,max_depth=80, max_features='sqrt', min_samples_split= 4, min_samples_leaf=3).fit(x_train,y_train)
  clf.fit(x_train,y_train)
  y_predicted = clf.predict(x_test)
  y_train_pred = clf.predict(x_train)
  pred_prob = clf.predict_proba(x_test)
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print('for PCA_Values =',i)
  print('Random forest classifier reults')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))
  print('--------------------------------')
  from xgboost import XGBRFClassifier
  xgb = XGBRFClassifier()
  xgb.fit(x_train,y_train)
  y_predicted = xgb.predict(x_test)
  y_train_pred = xgb.predict(x_train)
  pred_prob = xgb.predict_proba(x_test)
  print('XGBRClassifier results')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))
  print('-------------------------------')
  from lightgbm import LGBMClassifier
  lgb = LGBMClassifier()
  lgb.fit(x_train,y_train)
  y_predicted = lgb.predict(x_test)
  y_train_pred = lgb.predict(x_train)
  pred_prob = xgb.predict_proba(x_test)
  print('LGBMC_Classifier results')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))

------------------------------------------------------------------------
------------------------------------------------------------------------
------------------------------------------------------------------------
for PCA_Values = 2
Random forest classifier reults
Accuracy Test 83.91
Accuracy Train 96.00
ROC-AUC Score 85.76
Recall Score 94.12
--------------------------------
XGBRClassifier results
Accuracy Test 82.76
Accuracy Train 83.40
ROC-AUC Score 85.84
Recall Score 91.18
-------------------------------
LGBMC_Classifier results
Accuracy Test 81.61
Accuracy Train 100.00
ROC-AUC Score 85.84
Recall Score 88.24
------------------------------------------------------------------------
------------------------------------------------------------------------
------------------------------------------------------------------------
for PCA_Values = 3
Random forest classifier reults
Accuracy Test 80.46
Accuracy Train 96.20
ROC-AUC Score 85.76
Recall Score 89.71
--------------------------

#PCA on continuous dataset

In [100]:
for i in range(2,10):
  pca = PCA(n_components=i)
  principalComponents_i = pca.fit_transform(b)
  principalDf_i = pd.DataFrame(data = principalComponents_i)
  principalDf_i['6'] = a['6']
  principalDf_i['7'] = a['7']
  principalDf_i['8'] = a['8']
  principalDf_i['9'] = a['9']
  principalDf_i['10'] = a['10']
  principalDf_i['11'] = a['11']
  principalDf_i['12'] = a['12']
  principalDf_i['13'] = a['13']
  principalDf_i['14'] = a['14']
  principalDf_i['15'] = a['15']
  principalDf_i['16'] = a['16']
  principalDf_i['17'] = a['17']
  principalDf_i['18'] = a['18']
  principalDf_i['19'] = a['19']
  principalDf_i['20'] = a['20']
  principalDf_i['22'] = a['22']
  Y = Y.astype(int)
  Y = Y.reshape(-1)
  x_train, x_test, y_train, y_test = train_test_split(principalDf_i,Y,test_size = 0.148)
  scaler = preprocessing.StandardScaler().fit(x_train)
  x_train = scaler.transform(x_train)
  scaler = preprocessing.StandardScaler().fit(x_test)
  x_test = scaler.transform(x_test)
  from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
  n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
  max_features = ['auto', 'sqrt']
  max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
  max_depth.append(None)
  min_samples_split = [2, 3,5]
  min_samples_leaf = [ 2, 3, 4]
  bootstrap = [True, False]# Create the random grid
  random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
  clf = RandomForestClassifier(n_estimators=1333,bootstrap=True,max_depth=80, max_features='sqrt', min_samples_split= 4, min_samples_leaf=3).fit(x_train,y_train)
  clf.fit(x_train,y_train)
  y_predicted = clf.predict(x_test)
  y_train_pred = clf.predict(x_train)
  pred_prob = clf.predict_proba(x_test)
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print('------------------------------------------------------------------------')
  print('for PCA_Values =',i)
  print('Random forest classifier reults')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))
  print('--------------------------------')
  from xgboost import XGBRFClassifier
  xgb = XGBRFClassifier()
  xgb.fit(x_train,y_train)
  y_predicted = xgb.predict(x_test)
  y_train_pred = xgb.predict(x_train)
  pred_prob = xgb.predict_proba(x_test)
  print('XGBRClassifier results')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))
  print('-------------------------------')
  from lightgbm import LGBMClassifier
  lgb = LGBMClassifier()
  lgb.fit(x_train,y_train)
  y_predicted = lgb.predict(x_test)
  y_train_pred = lgb.predict(x_train)
  pred_prob = xgb.predict_proba(x_test)
  print('LGBMC_Classifier results')
  print("Accuracy Test %0.2f" %(accuracy_score(y_test,y_predicted)*100))
  print("Accuracy Train %0.2f" %(accuracy_score(y_train,y_train_pred)*100))
  print("ROC-AUC Score %0.2f" %(roc_auc_score(y_test,pred_prob[:,1])*100))
  print("Recall Score %0.2f" %(recall_score(y_test,y_predicted)*100))

------------------------------------------------------------------------
------------------------------------------------------------------------
------------------------------------------------------------------------
for PCA_Values = 2
Random forest classifier reults
Accuracy Test 73.56
Accuracy Train 88.80
ROC-AUC Score 73.73
Recall Score 92.54
--------------------------------
XGBRClassifier results
Accuracy Test 72.41
Accuracy Train 79.60
ROC-AUC Score 72.31
Recall Score 88.06
-------------------------------
LGBMC_Classifier results
Accuracy Test 75.86
Accuracy Train 98.40
ROC-AUC Score 72.31
Recall Score 89.55
------------------------------------------------------------------------
------------------------------------------------------------------------
------------------------------------------------------------------------
for PCA_Values = 3
Random forest classifier reults
Accuracy Test 72.41
Accuracy Train 89.80
ROC-AUC Score 69.16
Recall Score 92.31
---------------------------